# LangChain

- ChatPromptTemplate
## LCEL chaining
chain = prompt | llm | output_parser

In [1]:

!pip install -q langchain langchain-openai tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 837.6 kB/s eta 0:00:00


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
# OpenAI API 클라이언트 생성
API_KEY = os.getenv("OPENAI_API_KEY")

# # OpenAI API 키 설정 (실제 사용 시 본인의 API 키로 변경)
# os.environ["OPENAI_API_KEY"] = "your key"


In [2]:

from langchain_openai import ChatOpenAI

# model
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# chain 실행
result = llm.invoke("지구의 자전 주기는?")

In [3]:
result.content

'지구의 자전 주기는 약 24시간입니다. 즉, 지구는 한 번의 자전 주기를 완료하기 위해 약 24시간이 걸립니다.'

In [5]:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. : {input}")
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in astronomy. Answer the question. : {input}'), additional_kwargs={})])

In [6]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# chain 연결 (LCEL)
chain = prompt | llm

# chain 호출
chain.invoke({"input": "지구의 자전 주기는?"})

AIMessage(content='지구의 자전 주기는 약 24시간으로, 하루 동안 지구가 자기 축 주위를 한 바퀴 도는 시간을 말합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 28, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C8L03RCOVqTzeESFwE4ckAhnLLCbs', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--113d6d4f-c93d-4589-96c5-98a7e89577b0-0', usage_metadata={'input_tokens': 28, 'output_tokens': 48, 'total_tokens': 76, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# prompt + model + output parser
prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. : {input}")
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
output_parser = StrOutputParser()

# LCEL chaining
chain = prompt | llm | output_parser

# chain 호출
chain.invoke({"input": "지구의 자전 주기는?"})

'지구의 자전 주기는 약 24시간입니다. 이는 하루가 지나면 지구가 한 바퀴를 돌게 되는 시간을 의미합니다. 이것이 하루 밤과 낮이 나뉘는 이유이기도 합니다.'

# 멀티체인

In [8]:
prompt1 = ChatPromptTemplate.from_template("translates {korean_word} to English.")
prompt2 = ChatPromptTemplate.from_template(
    "explain {english_word} using oxford dictionary to me in Korean."
)

llm = ChatOpenAI(model="gpt-4o-mini")

chain1 = prompt1 | llm | StrOutputParser()

chain1.invoke({"korean_word":"미래"})


'The Korean word "미래" translates to "future" in English.'

In [10]:
chain2 = (
    {"english_word": chain1}
    | prompt2
    | llm
    | StrOutputParser()
)

chain2.invoke({"korean_word":"미래"})


'"미래"는 영어로 "future"를 의미합니다. 옥스포드 사전에 따르면, "future"는 \'어떤 사건이나 상황이 발생할 것으로 예상되는 시간\' 또는 \'앞으로 올 시간\'을 가리킵니다. 즉, "미래"는 우리가 아직 경험하지 않았지만 앞으로 일어날 수 있는 일들을 의미하는 단어입니다.'

# Runnable

In [12]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI # Changed import
from langchain.schema.output_parser import StrOutputParser

# 1. 컴포넌트 정의
prompt = ChatPromptTemplate.from_template("지구과학에서 {topic}에 대해 간단히 설명해주세요.")
model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

# 2. 체인 생성
chain = prompt | model | output_parser

# 3. invoke 메소드 사용
result = chain.invoke({"topic": "지구 자전"})
print("invoke 결과:", result)

invoke 결과: 지구 자전은 지구가 자신의 축을 중심으로 회전하는 과정을 말합니다. 이 회전은 지구의 자전축이 북극과 남극을 잇는 가상의 선을 기준으로 이루어지며, 약 24시간에 한 바퀴를 돈다. 지구 자전의 결과로 낮과 밤이 생기며, 일주일 주기와 계절의 변화에도 영향을 미칩니다. 

지구 자전은 또한 지구의 기후와 날씨 패턴, 조수 현상 등에도 중요한 역할을 하며, 자전 방향은 서쪽에서 동쪽으로 이루어집니다. 이로 인해 태양은 동쪽에서 떠서 서쪽으로 지는 것처럼 보입니다. 자전은 지구의 형성 초기부터 시작되어 현재까지 계속되고 있습니다.


In [13]:
# batch 메소드 사용
topics = ["지구 공전", "화산 활동", "대륙 이동"]
results = chain.batch([{"topic": t} for t in topics])
for topic, result in zip(topics, results):
    print(f"{topic} 설명: {result[:50]}...")  # 결과의 처음 50자만 출력


지구 공전 설명: 지구 공전(地球公轉)은 지구가 태양 주위를 타원형 궤도로 도는 운동을 의미합니다. 지구는 ...
화산 활동 설명: 화산 활동은 지구 내부의 마그마가 지각을 뚫고 나와 표면에 분출되는 현상을 말합니다. 화산...
대륙 이동 설명: 대륙 이동 이론은 지구의 대륙들이 시간이 지남에 따라 이동해왔다는 개념입니다. 이 이론은 ...


In [14]:
# stream 메소드 사용
stream = chain.stream({"topic": "지진"})
print("stream 결과:")
for chunk in stream:
    print(chunk, end="", flush=True)
print()


stream 결과:
지진은 지구 내부에서 발생하는 에너지 방출로 인해 지표면이 흔들리는 현상입니다. 주로 지각의 판이 움직이거나 서로 충돌, 분리, 스치면서 발생합니다. 이러한 판의 움직임은 응력(스트레스)을 축적하고, 이 응력이 한계를 넘어가게 되면 갑작스럽게 해소되면서 지진이 발생하게 됩니다.

지진의 발생 위치는 진원(지진의 발생 지점)과 지표에서의 위치인 진앙(진원의 수직선 위 위치)으로 구분됩니다. 지진의 강도는 리히터 규모나 모멘트 규모로 측정되며, 지진이 발생할 때 발생하는 파동은 주로 P파(압축파)와 S파(전단파)로 나뉩니다.

지진은 가끔 큰 피해를 초래할 수 있으며, 해저에서 발생하는 지진은 쓰나미를 유발할 수도 있습니다. 지진의 예측과 연구는 지구과학의 중요한 분야입니다.


In [15]:
import nest_asyncio
import asyncio

# nest_asyncio 적용 (구글 코랩 등 주피터 노트북에서 실행 필요)
nest_asyncio.apply()

# 비동기 메소드 사용 (async/await 구문 필요)
async def run_async():
    result = await chain.ainvoke({"topic": "해류"})
    print("ainvoke 결과:", result[:50], "...")

asyncio.run(run_async())


ainvoke 결과: 해류는 바다의 물이 지속적으로 흐르는 현상을 말합니다. 해류는 여러 요인에 의해 발생하며, ...


In [4]:
from langchain_core.prompts import PromptTemplate

# 'name'과 'age'라는 두 개의 변수를 사용하는 프롬프트 템플릿을 정의
template_text = "안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(name="홍길동", age=30)

filled_prompt

'안녕하세요, 제 이름은 홍길동이고, 나이는 30살입니다.'

In [ ]:
# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n아버지를 아버지라 부를 수 없습니다.")
              + "\n\n{language}로 번역해주세요."
)

combined_prompt
combined_prompt.format(name="홍길동", age=30, language="영어")

PromptTemplate(input_variables=['age', 'language', 'name'], input_types={}, partial_variables={}, template='안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n{language}로 번역해주세요.')